<a href="https://colab.research.google.com/github/abhishek-paghdal/Content_Generation/blob/main/this_is%20the_last_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.3 MB/s eta 0:00:00


###**Required Importing**

In [4]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
import os

###**Uploading Dataset**

In [5]:
df=pd.read_csv('/content/drive/MyDrive/Content_Project/data.csv')

In [6]:
df.head()

,_id,title,datePublished,description,article,keywords
0,{'$oid': '600837a2b5425d67582e0d8a'},EU lawmakers to push audio-visual sector on ge...,2020-11-30 05:46:35,European Union lawmakers are considering wheth...,European Union lawmakers are considering wheth...,"audio-visual, eu, geoblocking, Netflix, stream..."
1,{'$oid': '600837a3b5425d67582e0d8e'},Construction startup Scaled Robotics raises a ...,2020-02-03 07:13:18,"Industrial robots are expensive. But, then, so...","Industrial robots are expensive. But, then, so...","Battlefield, construction, Norwegian Construct..."
2,{'$oid': '600837a4b5425d67582e0d91'},Roblox buys digital avatar startup Loom.ai – T...,2020-12-14 11:10:15,Roblox announced today that it’s buying a digi...,Roblox announced today that it’s buying a digi...,"Avatar, Bitmoji, bitstrips, computing, films, ..."
3,{'$oid': '600837a4b5425d67582e0d92'},ClickUp hits $1 billion valuation in $100M Ser...,2020-12-15 16:30:14,Just six month after raising its first bit of ...,Just six month after raising its first bit of ...,"Android, ClickUp, Craft Ventures, Georgian Par..."
4,{'$oid': '600837a5b5425d67582e0d97'},"Spotify launches video podcasts worldwide, sta...",2020-07-21 07:13:39,Spotify today announced the global launch of v...,Spotify today announced the global launch of v...,"Spotify, streaming music, Video, YouTube"


##**Preprocessing Dataset**

In [7]:
#Checking if NULL rows are present
df.isna().sum()

_id                0
title              0
datePublished      0
description        0
article           21
keywords         919
dtype: int64

In [8]:
#Removing the row with null values
df.dropna(inplace=True)

In [9]:
#Making new dataset 
df1=df

In [10]:
df1.to_csv('data1.csv',index=False)

In [11]:
data=pd.read_csv('data1.csv')

In [12]:
data.shape

(24380, 6)

In [13]:
#Removing duplicates rows
data.drop_duplicates(inplace=True)

####**Now check the language of texts**

In [14]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=c41f06376a73a2cb968724f13877b2df8550ef16cb590bb8a59623d2e2f79841
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [15]:
import langdetect

In [16]:
def is_english(text):
    try:
        lang = langdetect.detect(text)
        if lang == 'en':
            return True
        else:
            return False
    except:
        return False

In [ ]:
data['is_english'] = data['article'].apply(lambda x: is_english(x))

In [ ]:
data.head()

In [ ]:
data['is_english'].value_counts()

In [ ]:
data= data[data['is_english']==True]

###**Text Length Count**

In [ ]:
from collections import Counter

In [ ]:
data['text_length'] = data['article'].apply(len)

In [ ]:
data.sort_values(by='text_length')

####**Now consider articles with length between (300 and 30000)**

In [ ]:
data= data[(data['text_length'] >= 300) & (data['text_length'] <= 20000)]

In [ ]:
data.shape

##**Split the dataset into train and evaluate**

In [ ]:
train_data, eval_data = train_test_split(data, test_size=0.2)

In [ ]:
train_data.shape

In [ ]:
train_data.to_csv('train_data.csv')
eval_data.to_csv('eval_data.csv')

In [ ]:
# this is for the train data

# Open the CSV file and read the contents
with open('/content/train_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/train_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


In [ ]:
# this is for the evaluate data

# Open the CSV file and read the contents
with open('/content/eval_data.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    data = [row['article'] for row in reader]

# Write the contents to a text file
with open('/content/eval_dataset.txt', 'w', encoding='utf-8') as txtfile:
    txtfile.write('\n'.join(data))


##**Load the model and tokenizer**

In [ ]:
# def load_model(model_dir):
#     tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#     model = GPT2LMHeadModel.from_pretrained('gpt2')

#     # Check if a model exists in the given directory
#     model_path = model_dir + "/model.pth"
#     if os.path.exists(model_path):
#         # Load the model
#         model.load_state_dict(torch.load(model_path))
#         print("Loaded model from:", model_path)
#     else:
#         print("No model found at:", model_path)

#     return model, tokenizer

In [ ]:
# model,tokenizer=load_model('/content/drive/MyDrive/GPT2_saved_model')

In [ ]:
# Load the pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
for name, param in model.named_parameters():
    if 'transformer.h.11' in name or 'transformer.h.10' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False


In [ ]:
for name, param in model.named_parameters():
    if not param.requires_grad:
        print(name)


In [ ]:
for name, child in model.named_children():
    print(name, 'requires_grad:', any(param.requires_grad for param in child.parameters()))



##**Prepare your dataset by creating a TextDataset object**

In [ ]:
# Prepare your dataset by creating a TextDataset object
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/train_dataset.txt',
    block_size=128
)

In [ ]:
# Prepare your dataset by creating a TextDataset object
eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path='/content/eval_dataset.txt',
    block_size=128
)


##**checkpoint using WANDB**

In [ ]:
pip install wandb

In [ ]:
import wandb
from transformers import Trainer, TrainingArguments
from transformers.integrations import WandbCallback

In [ ]:
# Set up WANB
wandb.login()
wandb.init(project="my_project_tech_articles")


##**Set up the trainer with the appropriate training arguments**

In [ ]:


# Define the training arguments
training_args = TrainingArguments(
    output_dir='.\results',
    evaluation_strategy='epoch',
    learning_rate=0.0001,
    per_device_train_batch_size=4,
    num_train_epochs=1,
    save_total_limit=2,
    save_steps=62,
    resume_from_checkpoint=True,
    weight_decay=0.001,
    report_to='wandb',  # Enable WandbCallback
   # load_best_model_at_end=True  # load best model checkpoint at the end
)

# Define the WandbCallback
wandb_callback = WandbCallback()

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[WandbCallback()],
)

##**Train the model**

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.set_grad_enabled(True)

In [1]:
trainer.train()

NameError: ignored

In [46]:
trainer.evaluate()

{'eval_loss': 3.5319020748138428,
 'eval_runtime': 6.9039,
 'eval_samples_per_second': 95.018,
 'eval_steps_per_second': 11.877,
 'epoch': 5.0}

In [ ]:
# # Save the best model checkpoint
# trainer.save_model('/path/to/save/directory')


###**Now code to rehit the promt to get required length article**

In [55]:

# Set the maximum length of each segment
max_segment_length = 1024

# Set the initial prompt
prompt = "Deep learning "

# Initialize the generated text
generated_text = prompt


input_ids = tokenizer.encode(prompt, return_tensors='pt')
 # Generate the next segment of text
output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             temperature=0.7,
                             top_k=50,
                             num_beams=5
                        
                             
    
    # Decode the output to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


# Print the generated text
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Deep learning  from Google’s new algorithm  of machine learning  about the human mind. This new algorithm will lead to a smarter AI,” that can better understand and understand human emotions,” and give us an accurate and accurate picture of what’s going on in our heads. The machine learning algorithms will be able to understand human emotions,” and predict their behavior,” even if they’re not in the right mood to do so. The algorithm will help us understand the emotions of our people,” and they will be able to use the algorithms to better inform us about their mental states. The AI will also learn how to use its information to help us in our daily lives,” and it will learn to distinguish between emotion and information in a more intuitive way. For instance, the AI will be able to learn to distinguish between a human’s words,” and a computer’s action,” and also to distinguish between different kinds of emotions.” These algorithms will be able to predict which emotions are most important

In [52]:

# Set the maximum length of each segment
max_segment_length = 512

# Set the initial prompt
prompt = "Tesla automated vehicles"

# Initialize the generated text
generated_text = prompt

# Generate segments of text and stitch them together
while len(generated_text) < 500:
    # Encode the prompt as input IDs
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate the next segment of text
    output = model.generate(input_ids=input_ids.to('cuda'),
                             max_length=max_segment_length,
                             do_sample=True,
                             top_p=0.95,
                             temperature=0.7,
                             top_k=50)
    
    # Decode the output to text
    segment = tokenizer.decode(output[0], skip_special_tokens=True)

    # Add the segment to the generated text
    generated_text += segment

    # Get the last 1024 tokens of the previous segment as the new prompt
    prompt = generated_text[-max_segment_length:]

# Print the generated text
print(generated_text)
final_text=generated_text


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Tesla automated vehiclesTesla automated vehicles on public roads, such as public transportation and public transit, and public transport systems, such as taxis and Uber, have been a key driver of the industry, the company said in a statement. The company said it will continue to invest in autonomous vehicles, which it is calling a “high-growth opportunity,” said the announcement. “The company said it expects to raise more capital this year than in the past, noting that it expects to increase its capital expenditures by 10% this year. “The company said it expects to make $4.2 billion in capital expenditures this year, up from $2.7 billion in the same period last year, while it expects to raise $8.9 billion in capital expenditures this year, down from $9.1 billion in the same period in the same period last year. It also expects to invest more than $5 billion in the next three years, up from $4.7 billion in the same period last year.” The new investments in autonomous vehicles, which it s

In [ ]:
print(len(final_text))

1592


In [ ]:
type(final_text)

str

##**Use text summarization to improve article**

In [ ]:
# Install required libraries
#!pip install transformers

# Import required libraries
from transformers import pipeline

# Load pre-trained summarization model
#summarizer = pipeline("summarization")
# Load pre-trained summarization model (using BART-Large-CNN)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


# Define input text to summarize
input_text = "The United States is a country located in North America. It is the third largest country in the world by total area and the third most populous country in the world. The United States has a diverse geography and climate, with mountains, forests, deserts, and coastal areas. Its economy is the largest in the world, driven by industries such as technology, finance, and healthcare."

# Generate summary for input text
summary = summarizer(final_text, max_length=1000, min_length=30, do_sample=False)

# Print generated summary
print(summary[0]['summary_text'])


Your max_length is set to 1000, but you input_length is only 304. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=152)


New tools are being made available to developers through the “Digital Machines’ platform, which is open source and accessible to the public. Developers can “use the Digital Machines” platform to connect with a number of existing and emerging AI capabilities.


##**Below code is for save the model**

In [ ]:
# model.to('cuda')a# Convert the model to CPU

model = model.to(torch.device('cpu'))


In [ ]:

# Generate text using the fine-tuned model
prompt = 'which is the best phone that i can buy ?'
generated_text = model.generate(
    input_ids=tokenizer.encode(prompt, return_tensors='pt'),
    max_length=200,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
)

print(tokenizer.decode(generated_text[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


which is the best phone that i can buy? ) and other  Android based devices  in a year, it also’s a fairly large portfolio that also features more than 100,000 square feet of available office space. The company offers the new HomePod speaker, Alexa speaker and Alexa-powered streaming and  cloud services  in addition to the usual smart home accessories and accessories. In terms of the product, it’s not really great, though it’s a small handset that  packs in some pretty nice battery life. The only real complaint I have with the phone is its camera, which the company did provide in an in-person demo. There is a built-in Wi-Fi hotspot to minimize the risk that you might connect to a neighboring area but there are no hotspots available on the device. However, with its smaller screen size, the HomePod speaker isn’t far behind the competition, and if you want to bring a full pair of


In [ ]:
# trainer.save_model('Second_Model_GPT2-Epoch_3_1000')

In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile("Model_2_GPT-2_Epoch_3_1000.zip", "r")
# zip_ref.extractall()
# zip_ref.close()

In [ ]:
import joblib

# joblib.dump(model, 'SecondModel_Epoch_3_1000_2.joblib')

In [ ]:


joblib.dump(model, 'Model_Epoch_5_3000.joblib')

['Model_Epoch_5_3000.joblib']

In [ ]:
joblib.dump(tokenizer, 'tokenizer_5_3000.joblib')


['tokenizer_5_3000.joblib']

In [ ]:
loaded_model_joblib=joblib.load('SecondModel_Epoch_3_1000_2.joblib')

FileNotFoundError: ignored

In [ ]:
loaded_model_joblib

In [ ]:
loaded_model_joblib

In [ ]:
import joblib as jb

In [ ]:
jb.dump(model,'model_jb.pkl')

In [ ]:
loaded_model = jb.load('model_jb.pkl')